In [ ]:
from pyparsing import Word, alphas


In [ ]:
greet = Word( alphas ) + "," + Word( alphas ) + "!" # <-- grammar defined here
greet1 = ( Word( alphas ) + "," + Word( alphas ) + "!") | ("!" + Word(alphas) + "!")

In [ ]:
hello = "Hello, World!"
hello2 = "! Hello !"
hello3 = "! Hello, Here !"
print (hello2, "->", greet1.parseString( hello2 ))

In [ ]:
# chemicalFormulas.py
from pyparsing import Word, Optional, OneOrMore, Group, ParseException

# define a simple Python dict of atomic weights, with chemical symbols
# for keys
atomicWeight = {
    "O"  : 15.9994,
    "H"  : 1.00794,
    "Na" : 22.9897,
    "Cl" : 35.4527,
    "C"  : 12.0107,
    "S"  : 32.0655,
    }

# define some strings to use later, when describing valid lists 
# of characters for chemical symbols and numbers
caps = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
lowers = caps.lower()
digits = "0123456789"

# Version 1
# Define grammar for a chemical formula
# - an element is a Word, beginning with one of the characters in caps,
#   followed by zero or more characters in lowers
# - an integer is a Word composed of digits
# - an elementRef is an element, optionally followed by an integer - if 
#   the integer is omitted, assume the value "1" as a default; these are 
#   enclosed in a Group to make it easier to walk the list of parsed 
#   chemical symbols, each with its associated number of atoms per 
#   molecule
# - a chemicalFormula is just one or more elementRef's
element = Word( caps, lowers )
integer = Word( digits )
elementRef = Group( element + Optional( integer, default="1" ) )
chemicalFormula = OneOrMore( elementRef )

In [ ]:
# try parsing some simple formulas
for formula in ( "H2O", "NaCl", "C6H5OH", "H2SO4" ):
    formulaData = chemicalFormula.parseString(formula)
    
    # compute molecular weight
    # - the following Python expression is a shorthand for this for loop
    #   mw = 0
    #   for sym,qty in formulaData:
    #       mw += atomicWeight[sym]*int(qty)
    mw = sum( [ atomicWeight[sym]*int(qty) for sym,qty in formulaData ] )

    # print the results
    print(formula, "->", formulaData, "(", mw, ")")

print()

In [8]:
from __future__ import division
from pyparsing import (Literal, CaselessLiteral, Word, Combine, Group, Optional,
                       ZeroOrMore, Forward, nums, alphas, oneOf)
import math
import operator

__author__ = 'Paul McGuire'
__version__ = '$Revision: 0.0 $'
__date__ = '$Date: 2009-03-20 $'
__source__ = '''http://pyparsing.wikispaces.com/file/view/fourFn.py
http://pyparsing.wikispaces.com/message/view/home/15549426
'''
__note__ = '''
All I've done is rewrap Paul McGuire's fourFn.py as a class, so I can use it
more easily in other places.
'''


class NumericStringParser(object):
    '''
    Most of this code comes from the fourFn.py pyparsing example

    '''

    def pushFirst(self, strg, loc, toks):
        self.exprStack.append(toks[0])

    def pushUMinus(self, strg, loc, toks):
        if toks and toks[0] == '-':
            self.exprStack.append('unary -')

    def __init__(self):
        """
        expop   :: '^'
        multop  :: '*' | '/'
        addop   :: '+' | '-'
        integer :: ['+' | '-'] '0'..'9'+
        atom    :: PI | E | real | fn '(' expr ')' | '(' expr ')'
        factor  :: atom [ expop factor ]*
        term    :: factor [ multop factor ]*
        expr    :: term [ addop term ]*
        """
        point = Literal(".")
        e = CaselessLiteral("E")
        fnumber = Combine(Word("+-" + nums, nums) +
                          Optional(point + Optional(Word(nums))) +
                          Optional(e + Word("+-" + nums, nums)))
        ident = Word(alphas)
        array_idx = Word("$" + nums)
        array_all = Literal("$$")
        plus = Literal("+")
        minus = Literal("-")
        mult = Literal("*")
        div = Literal("/")
        lpar = Literal("(").suppress()
        rpar = Literal(")").suppress()
        addop = plus | minus
        multop = mult | div
        expop = Literal("^")
        pi = CaselessLiteral("PI")
        expr = Forward()
        atom = ((Optional(oneOf("- +")) +
                 (ident + lpar + expr + rpar | pi | e | fnumber).setParseAction(self.pushFirst))
                | ( Optional(oneOf("- +")) + Group(lpar + expr + rpar) )
                | array_idx | array_all
                ).setParseAction(self.pushUMinus) 
        # by defining exponentiation as "atom [ ^ factor ]..." instead of
        # "atom [ ^ atom ]...", we get right-to-left exponents, instead of left-to-right
        # that is, 2^3^2 = 2^(3^2), not (2^3)^2.
        factor = Forward()
        factor << atom + \
            ZeroOrMore((expop + factor).setParseAction(self.pushFirst))
        term = factor + \
            ZeroOrMore((multop + factor).setParseAction(self.pushFirst))
        expr << term + \
            ZeroOrMore((addop + term).setParseAction(self.pushFirst))
        # addop_term = ( addop + term ).setParseAction( self.pushFirst )
        # general_term = term + ZeroOrMore( addop_term ) | OneOrMore( addop_term)
        # expr <<  general_term
        self.bnf = expr
        
        # map operator symbols to corresponding arithmetic operations
        epsilon = 1e-12
        self.opn = {"+": operator.add,
                    "-": operator.sub,
                    "*": operator.mul,
                    "/": operator.truediv,
                    "^": operator.pow}
        self.fn = {"sin": math.sin,
                   "cos": math.cos,
                   "tan": math.tan,
                   "exp": math.exp,
                   "abs": abs,
                   "trunc": lambda a: int(a),
                   "round": round,
                   "sgn": lambda a: abs(a) > epsilon and cmp(a, 0) or 0}

    def evaluateStack(self, s):
        op = s.pop()
        print("op = ", op)
        if op == 'unary -':
            return -self.evaluateStack(s)
        if op in "+-*/^":
            op2 = self.evaluateStack(s)
            op1 = self.evaluateStack(s)
            return self.opn[op](op1, op2)
        elif op == "PI":
            return math.pi  # 3.1415926535
        elif op == "E":
            return math.e  # 2.718281828
        elif op[0] == "$":
            return float(op[1:])
        elif op in self.fn:
            return self.fn[op](self.evaluateStack(s))
        elif op[0].isalpha():
            return 0
        else:
            return float(op)

    def eval(self, num_string, parseAll=True):
        self.exprStack = []
        # parsing
        results = self.bnf.parseString(num_string, parseAll)
        print("$ parse results:\n", results)
        # evaluation
        val = self.evaluateStack(self.exprStack)
        return val

In [9]:
nsp = NumericStringParser()
result = nsp.eval('+ 2^4')
print(result)
# 16.0

result = nsp.eval('exp(2^4)')
print(result)
# 8886110.520507872

result = nsp.eval('1+ 2 + 1 + 0')
print(result)

$ parse results:
 ['+', '2', '^', '4']
op =  ^
op =  4
op =  2
16.0
$ parse results:
 ['exp', '2', '^', '4']
op =  exp
op =  ^
op =  4
op =  2
8886110.520507872
$ parse results:
 ['1', '+', '2', '+', '1', '+', '0']
op =  +
op =  0
op =  +
op =  1
op =  +
op =  2
op =  1
4.0
